# Member Transaction Balance Analysis

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pd.options.plotting.backend = 'plotly'

from IPython.display import display

# DSC 80 preferred styles
pio.templates["dsc80"] = go.layout.Template(
    layout=dict(
        margin=dict(l=30, r=30, t=30, b=30),
        autosize=True,
        width=600,
        height=400,
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
        title=dict(x=0.5, xanchor="center"),
    )
)
pio.templates.default = "simple_white+dsc80"

import plotly.figure_factory as ff
def create_kde_plotly(df, group_col, group1, group2, vals_col, title=''):
    fig = ff.create_distplot(
        hist_data=[df.loc[df[group_col] == group1, vals_col], df.loc[df[group_col] == group2, vals_col]],
        group_labels=[group1, group2],
        show_rug=False, show_hist=False
    )
    return fig.update_layout(title=title)

### First, import the data from the correct csv file and store it in a DataFrame

In [2]:
financial_path = Path('financial_data.csv')
financial_data = pd.read_csv(financial_path)
financial_data

,customer,location,initial_date,transaction_date,amount
0,cust00003,22,2023-06-01,2023-06-01,39.00
1,cust00015,22,2023-03-01,2023-03-01,39.00
2,cust00016,22,2023-08-01,2023-08-01,39.00
3,cust00030,22,2023-08-01,2023-08-01,39.00
4,cust00044,22,2023-08-01,2023-08-01,39.00
...,...,...,...,...,...
11789,cust99885,22,2023-06-01,2023-06-01,19.99
11790,cust99905,22,2023-01-01,2023-01-01,39.00
11791,cust99910,22,2023-10-01,2023-10-01,10.00
11792,cust99911,22,2023-10-01,2023-10-01,10.00


## Part 1. Declined Transactions

For our analysis, we are interested in the members whose initial transaction was declined. So, for this part, we will look at a few main things:
- What percentage of members had their initial transaction declined? This will be the proportion of members in the DataFrame who appear in more than one row.
- Of the members who had their initial transaction declined, how many ended up successfully being charged (As a proportion), and how much money was this in total?
- Of the members who had their initial transaction declined, how many were never successfully charged (As a proportion), and how much money is then missing in total?

We can begin with this, and perform more sophisticated analysis after our initial discoveries.

In [3]:
#Set up correct DataFrame

copy_df = financial_data.copy()

#Calling .filter() on a GroupBy object returns an ungrouped DataFrame, filtered by groups that
#satisfy the lambda function. (x["amount"] < 0).count_nonzero > 0 filters by all groups with at least one row with a negative charge value. In other words,
#this filters the financial_data DataFrame so that it only contains information about members who were refunded money at some point:
#that is, their initial transaction was declined.
row_count = copy_df.groupby("customer").filter(lambda x: np.count_nonzero(np.array((x["amount"] < 0))) > 0)
row_count



,customer,location,initial_date,transaction_date,amount
34,cust00312,22,2023-07-01,2023-07-20,-22.06
35,cust00312,22,2023-07-01,2023-07-28,39.00
36,cust00312,22,2023-07-01,2023-07-17,22.06
46,cust00402,22,2023-08-01,2023-08-25,10.00
47,cust00402,22,2023-09-01,2023-09-06,-39.00
...,...,...,...,...,...
11613,cust97119,22,2023-09-01,2023-09-17,19.99
11614,cust97119,22,2023-09-01,2023-09-22,19.99
11724,cust98913,22,2023-02-01,2023-02-01,19.99
11725,cust98913,22,2023-02-01,2023-02-10,19.99


In [4]:
#Proportion of members whose initial transaction was declined

num_declined = row_count.groupby("customer").count().shape[0]
num_total = copy_df.groupby("customer").count().shape[0]

print(num_total)
print(num_declined)

prop_declined = num_declined / num_total
prop_declined

10574
451


0.04265178740306412

## Calculating the proportion of successfully charged

#### The question of how many people ended up being successfully charged is more complicated.



For example,

In [5]:
row_count[row_count["customer"] == "cust32798"]

,customer,location,initial_date,transaction_date,amount
8910,cust32798,22,2023-07-01,2023-07-24,-25.05
8911,cust32798,22,2023-07-01,2023-07-21,25.05
8912,cust32798,22,2023-08-01,2023-08-02,-49.00
8913,cust32798,22,2023-07-01,2023-07-18,-25.05
8914,cust32798,22,2023-08-01,2023-08-01,49.00
8915,cust32798,22,2023-07-01,2023-07-17,25.05


#### What the hell is that?

Let's work to figure this out.

In [6]:
#Convert contract_date to datetime object to separate charges by month

date_format = '%Y-%m-%d'
row_count["initial_date"] = pd.to_datetime(row_count['initial_date'], format=date_format)
row_count["month"] = row_count["initial_date"].dt.month

In [7]:
#Of the members who had their initial transaction declined, how many ended up successfully being charged?

def custom_helper(amount_series):
    amount_array = np.array(amount_series)

    visited_amounts = np.array([])

    #Variables to return
    outstanding_balance = 0
    num_failed_transactions = 0
    irregularity = ""
    
    #Loop through amounts in the array
    for amount in amount_array:
        #If the amount is negative, continue
        if amount < 0:
            if (amount * -1) in amount_array:
                continue
            else:
                irregularity += "Erroneous refund/Month error"

        #If the amount has already been visited, continue onward
        if amount not in visited_amounts:
            attempted_transactions = np.count_nonzero(amount_array == amount)
            num_failed_transactions += np.count_nonzero((amount_array == (amount * -1)))

            #Outstanding balance assumes no irregularities. These are handled later.
            outstanding_balance += (attempted_transactions - num_failed_transactions == 0) * amount

            if num_failed_transactions > attempted_transactions:
                irregularity += "Erroneous refund/Month error"

            #Append amount to visited amounts.
            visited_amounts = np.append(visited_amounts, amount)
    
    return list([num_failed_transactions, outstanding_balance, irregularity])



grouped_custom_helper = row_count.groupby(["customer", "month"])[["amount"]].agg(custom_helper)


In [8]:

#Extract Data from "amount" column

grouped_custom_helper.loc[:, 'Failed Transactions'] = grouped_custom_helper.amount.map(lambda x: x[0])
grouped_custom_helper.loc[:, 'Outstanding Balance'] = grouped_custom_helper.amount.map(lambda x: x[1])
grouped_custom_helper.loc[:, 'Irregularities'] = grouped_custom_helper.amount.map(lambda x: x[2])

grouped = grouped_custom_helper.drop(columns = "amount")

## Remember, we're here to calculate the proportion of people who had their initial transaction declined, and ended up successfully being charged.

In [9]:
grouped

Failed Transactions  Outstanding Balance  \
customer  month                                             
cust00312 7                        1                22.06   
cust00402 8                        2                10.00   
          9                        1                39.00   
cust01684 9                        2                10.00   
          10                       0                 0.00   
...                              ...                  ...   
cust94730 7                        0                 0.00   
cust96562 3                        1                 0.00   
cust97119 9                        2                19.99   
          10                       1                19.99   
cust98913 2                        1                 0.00   

                               Irregularities  
customer  month                                
cust00312 7                                    
cust00402 8                                    
          9                                    
cust01684 9                                    
          10                                   
...                                       ...  
cust94730 7      Erroneous refund/Month error  
cust96562 3                                    
cust97119 9                                    
          10                                   
cust98913 2                                    

[550 rows x 3 columns]

In [10]:
grouped_reset = grouped.reset_index()

In [11]:
grouped_reset.groupby("customer").sum().sort_values(by="Outstanding Balance", ascending=False)

,month,Failed Transactions,Outstanding Balance
customer,,,
cust32798,15,3,74.05
cust33641,21,4,74.05
cust33054,17,3,74.05
cust33535,21,3,74.05
cust32972,17,3,74.05
...,...,...,...
cust32875,8,1,0.00
cust32886,8,1,0.00
cust29219,5,1,0.00


In [12]:
outstanding_balances = grouped_reset.groupby("customer").sum().sort_values(by="Outstanding Balance")
proportion_successful = outstanding_balances[outstanding_balances["Outstanding Balance"] == 0].shape[0] / outstanding_balances.shape[0]
proportion_successful

0.3968957871396896

# Cool, 
#### but doesn't take into consideration the fact that some people may have been refunded way after their transaction was initially charged. Maybe this doesn't change anything, but at least it's something to be aware of.

For example,

In [13]:
row_count[row_count["customer"] == "cust94730"]

,customer,location,initial_date,transaction_date,amount,month
11449,cust94730,22,2023-07-01,2023-07-19,-39.0,7
11450,cust94730,22,2023-06-01,2023-06-01,39.0,6


#### As you can see, this dude's credit card bounced 6 weeks after it was originally charged. Here is a list of people with these sort of irregularities in their Credit Card activity:

In [14]:
grouped[grouped["Irregularities"] == "Erroneous refund/Month error"]

,,Failed Transactions,Outstanding Balance,Irregularities
customer,month,,,
cust20694,10,0,0.0,Erroneous refund/Month error
cust23505,4,0,0.0,Erroneous refund/Month error
cust24412,8,0,0.0,Erroneous refund/Month error
cust24496,4,0,0.0,Erroneous refund/Month error
cust29234,3,1,10.0,Erroneous refund/Month error
cust31093,3,0,0.0,Erroneous refund/Month error
cust31442,4,0,0.0,Erroneous refund/Month error
cust33923,12,0,0.0,Erroneous refund/Month error
cust94730,7,0,0.0,Erroneous refund/Month error


If we want to, we could go into each one of these member's data and look at what triggered our function to alert us to an irregularity. For example,

In [15]:
row_count[row_count["customer"] == "cust20694"]

,customer,location,initial_date,transaction_date,amount,month
2590,cust20694,22,2023-11-01,2023-11-17,22.06,11
2591,cust20694,22,2023-12-01,2023-12-17,16.94,12
2592,cust20694,22,2023-10-01,2023-10-24,-39.00,10


What the hell is going on with that dude?
<br>
<br>
There are other instances of stuff like this too. Here is another archetype, repeated from above:

In [16]:
row_count[row_count["customer"] == "cust94730"]

,customer,location,initial_date,transaction_date,amount,month
11449,cust94730,22,2023-07-01,2023-07-19,-39.0,7
11450,cust94730,22,2023-06-01,2023-06-01,39.0,6


In this case, the member was correctly refunded their money, but because they paid in June and were refunded in July, our function flags it as an irregularity. Which it might be. This is by design.

<br>
<br>

These are instances where wonky stuff is going on with the amounts being charged/refunded to people's credit cards. Other than that, our DataFrame contains the outstanding balances of each individual.

## How many people, and how much money?

#### Now, let's calculate the amount of outstanding money in tis DataFrame.

In [17]:
outstanding_balances

,month,Failed Transactions,Outstanding Balance
customer,,,
cust98913,2,1,0.00
cust27337,4,1,0.00
cust27379,4,1,0.00
cust32603,7,2,0.00
cust32457,7,1,0.00
...,...,...,...
cust32520,13,3,74.05
cust33535,21,3,74.05
cust32742,15,3,74.05


In [18]:
money_money_money = outstanding_balances[outstanding_balances["Outstanding Balance"] > 0]

In [19]:
money_money_money["Outstanding Balance"].describe()

count    272.000000
mean      26.626029
std       15.814579
min       10.000000
25%       22.060000
50%       25.050000
75%       25.050000
max       74.050000
Name: Outstanding Balance, dtype: float64

Cool stuff. Important to note that this exlcudes people whose credit card initially bounced and then were eventually charged. Additionally, this is an estimated value, that does not take into consideration erroneous refunds, month-by-month descrepancies, and other potential irregularities.